In [ ]:
# Import required libraries  
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
# True parameters
gamma_true = 0.1
omega_true = 1.0

def damped_oscillator(t, y, gamma, omega):
    x, x_dot = y
    x_ddot = -2*gamma*x_dot - omega**2*x
    return np.array([x_dot, x_ddot])

# Initial conditions
x0 = 1.0
xdot0 = 0.0
y0 = np.array([x0, xdot0])

# Time domain
t_span = (0, 10)
t_eval = np.linspace(0, 10, 100)

# Solve the ODE
sol = solve_ivp(damped_oscillator, t_span, y0, args=(gamma_true, omega_true), 
                t_eval=t_eval, method='RK45', rtol=1e-8)

# Extract solution
t_data = sol.t
x_data = sol.y[0]

# Plot ground truth
plt.figure(figsize=(10, 4))
plt.plot(t_data, x_data, 'b-', linewidth=2, label='Ground truth')
plt.xlabel('Time t')
plt.ylabel('x(t)')
plt.title('Ground Truth: Damped Oscillator')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

print(f"Generated {len(t_data)} data points")
# Define fully-connected neural network
class FCN(nn.Module):
    
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)
    
    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Define neural network
pinn = FCN(1, 1, 32, 3)  # Input: t, Output: x(t), Hidden: 32 nodes, 3 layers

# Define trainable parameters gamma and omega
gamma = torch.nn.Parameter(torch.tensor([0.5], requires_grad=True))  # Initial guess
omega = torch.nn.Parameter(torch.tensor([0.5], requires_grad=True))  # Initial guess

print("PINN Architecture:")
print(pinn)
print(f"\nTrainable parameters:")
print(f"  - gamma (initial): {gamma.item():.4f}")
print(f"  - omega (initial): {omega.item():.4f}")

# Prepare training data
# Number of data points to use for training 
n_data = 20  # Use 20 data points
data_indices = np.linspace(0, len(t_data)-1, n_data, dtype=int)
t_data_train = torch.tensor(t_data[data_indices], dtype=torch.float32).view(-1, 1)
x_data_train = torch.tensor(x_data[data_indices], dtype=torch.float32).view(-1, 1)

# Number of collocation points for physics loss
n_collocation = 100
t_physics = torch.linspace(0, 10, n_collocation).view(-1, 1).requires_grad_(True)

# Initial condition points
t_ic = torch.tensor([[0.0]], requires_grad=True)

# Test points for evaluation
t_test = torch.linspace(0, 10, 200).view(-1, 1)

print(f"Training setup:")
print(f"  - Number of data points: {n_data}")
print(f"  - Number of collocation points: {n_collocation}")
print(f"  - Time domain: [0, 10]")

# Setup optimizer (include PINN parameters and trainable gamma, omega)
optimizer = torch.optim.Adam(list(pinn.parameters()) + [gamma, omega], lr=1e-3)

# Hyperparameters for loss weighting
lambda_data = 1.0      # Weight for data loss
lambda_physics = 1e-4  # Weight for physics loss
lambda_ic = 1.0        # Weight for initial conditions

# Training loop
n_epochs = 20000
loss_history = []
gamma_history = []
omega_history = []

print("Starting training...")
for epoch in range(n_epochs):
    optimizer.zero_grad()
    
    # 1. Data loss: MSE between PINN prediction and observed data
    x_pred_data = pinn(t_data_train)
    loss_data = torch.mean((x_pred_data - x_data_train)**2)
    
    # 2. Physics loss: Residual of the ODE on collocation points
    x_pred_physics = pinn(t_physics)
    x_t = torch.autograd.grad(x_pred_physics, t_physics, 
                              torch.ones_like(x_pred_physics), 
                              create_graph=True)[0]
    x_tt = torch.autograd.grad(x_t, t_physics, 
                               torch.ones_like(x_t), 
                               create_graph=True)[0]
    
    # ODE residual: x'' + 2*gamma*x' + omega^2*x = 0
    residual = x_tt + 2*gamma*x_t + omega**2*x_pred_physics
    loss_physics = torch.mean(residual**2)
    
    # 3. Initial condition loss
    x_ic = pinn(t_ic)
    x_ic_t = torch.autograd.grad(x_ic, t_ic, 
                                  torch.ones_like(x_ic), 
                                  create_graph=True)[0]
    loss_ic_x = (x_ic - 1.0)**2  # x(0) = 1
    loss_ic_xdot = (x_ic_t - 0.0)**2  # x'(0) = 0
    loss_ic = loss_ic_x + loss_ic_xdot
    
    # Total loss
    loss = lambda_data*loss_data + lambda_physics*loss_physics + lambda_ic*loss_ic
    
    # Backpropagation
    loss.backward()
    optimizer.step()
    
    # Record history
    loss_history.append(loss.item())
    gamma_history.append(gamma.item())
    omega_history.append(omega.item())
    
    # Print progress
    if (epoch + 1) % 2000 == 0:
        print(f"Epoch {epoch+1}/{n_epochs} | Loss: {loss.item():.6f} | "
              f"γ: {gamma.item():.4f} | ω: {omega.item():.4f}")

print("\nTraining complete!")
print(f"Final γ: {gamma.item():.6f} (true: {gamma_true})")
print(f"Final ω: {omega.item():.6f} (true: {omega_true})")

# Plot training history
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Total loss
axes[0].plot(loss_history, 'b-', linewidth=1)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Total Loss')
axes[0].set_title('Training Loss')
axes[0].set_yscale('log')
axes[0].grid(True, alpha=0.3)

# Plot 2: Gamma convergence
axes[1].plot(gamma_history, 'r-', linewidth=1, label='PINN estimate')
axes[1].axhline(y=gamma_true, color='g', linestyle='--', linewidth=2, label='True value')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('γ')
axes[1].set_title('Convergence of γ')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: Omega convergence
axes[2].plot(omega_history, 'r-', linewidth=1, label='PINN estimate')
axes[2].axhline(y=omega_true, color='g', linestyle='--', linewidth=2, label='True value')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('ω')
axes[2].set_title('Convergence of ω')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Generate predictions
with torch.no_grad():
    x_pred = pinn(t_test).numpy()

# Compute errors
gamma_error = abs(gamma.item() - gamma_true)
omega_error = abs(omega.item() - omega_true)
gamma_rel_error = 100 * gamma_error / gamma_true
omega_rel_error = 100 * omega_error / omega_true

# Compute solution error (MSE on full ground truth)
t_full = torch.tensor(t_data, dtype=torch.float32).view(-1, 1)
with torch.no_grad():
    x_pred_full = pinn(t_full).numpy()
x_true_full = x_data
mse_solution = np.mean((x_pred_full.flatten() - x_true_full)**2)
rmse_solution = np.sqrt(mse_solution)

print("=" * 60)
print("EVALUATION RESULTS")
print("=" * 60)
print("\nParameter Recovery:")
print(f"  γ (gamma):")
print(f"    True value:      {gamma_true:.6f}")
print(f"    PINN estimate:   {gamma.item():.6f}")
print(f"    Absolute error:  {gamma_error:.6f}")
print(f"    Relative error:  {gamma_rel_error:.2f}%")
print(f"\n  ω (omega):")
print(f"    True value:      {omega_true:.6f}")
print(f"    PINN estimate:   {omega.item():.6f}")
print(f"    Absolute error:  {omega_error:.6f}")
print(f"    Relative error:  {omega_rel_error:.2f}%")

print(f"\nSolution Accuracy:")
print(f"  RMSE on full domain: {rmse_solution:.6f}")
print(f"  MSE on full domain:  {mse_solution:.6f}")

print("\n" + "=" * 60)
print("DISCUSSION")
print("=" * 60)
print("\nDiscrepancies:")
if gamma_rel_error < 5 and omega_rel_error < 5:
    print("  - Both parameters recovered with excellent accuracy (<5% error)")
elif gamma_rel_error < 10 and omega_rel_error < 10:
    print("  - Both parameters recovered with good accuracy (<10% error)")
else:
    print("  - Some discrepancies observed in parameter recovery")

if gamma_rel_error > omega_rel_error:
    print(f"  - γ has larger relative error ({gamma_rel_error:.2f}%) than ω")
else:
    print(f"  - ω has larger relative error ({omega_rel_error:.2f}%) than γ")



plt.figure(figsize=(12, 5))
plt.plot(t_data, x_data, 'b-', linewidth=3, label='Ground truth', alpha=0.7)
plt.plot(t_test.numpy(), x_pred, 'r--', linewidth=2, label='PINN prediction')
plt.scatter(t_data_train.numpy(), x_data_train.numpy(), 
            c='green', s=80, zorder=5, marker='o', 
            edgecolors='darkgreen', linewidth=1.5,
            label=f'Training data (n={n_data})', alpha=0.8)
plt.xlabel('Time t', fontsize=13)
plt.ylabel('x(t)', fontsize=13)
plt.title(f'PINN Solution: γ={gamma.item():.4f} (true: {gamma_true}), ω={omega.item():.4f} (true: {omega_true})', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=11, loc='upper right')
plt.grid(True, alpha=0.3)
plt.xlim(0, 10)
plt.show()

